# This script is mimicking what the backend would be doing when running an UDF

In [1]:
from os import getcwd
getcwd()

'/home/pzellner@eurac.edu/git_projects/r4openeo-usecases/uc2-ts-breakdetection/local_udf'

In [2]:
from openeo_r_udf.udf_lib import prepare_udf, execute_udf
import time
import xarray as xr

In [3]:
# config
#load_file = "../00_data_local/test.nc/openEO.nc"
load_file = "../00_data_local/vaja.nc/openEO.nc"
#load_file = "../00_data_local/test_eurac.nc"

#save_file = "./results/local_udf_test.nc"
save_file = "./results/local_udf_vaja.nc"


udf_file = "../00_udfs/bfast_udf.R"

dimension = "t"
context = {'start_monitor': 2018}

In [4]:
# load and prepare data - depending on your data structure
#dataset_vito = xr.open_dataset(load_file, engine = 'h5netcdf', decode_coords = 'all')
dataset = xr.open_dataset(load_file, engine = 'h5netcdf', decode_coords = 'all')

In [5]:
#dataset = dataset.rename({"time":"t"})
dataset

<xarray.Dataset>
Dimensions:  (t: 525, x: 2269, y: 2697)
Coordinates:
  * t        (t) datetime64[ns] 2016-11-05 2016-11-08 ... 2020-12-29 2021-01-01
  * x        (x) float64 6.785e+05 6.785e+05 6.785e+05 ... 7.011e+05 7.011e+05
  * y        (y) float64 5.152e+06 5.152e+06 5.152e+06 ... 5.125e+06 5.125e+06
    crs      |S1 ...
Data variables:
    var      (t, y, x) float32 ...
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.6.5a1
    description:  
    title:

In [6]:
# define runer for udf
def run(process, udf, data = None, dimension = None, context = None):
    # Run UDF executor
    t1 = time.time() # Start benchmark
    #print('start: ', udf, ' at ', time.strftime('%X %x %Z')) #t1
    result = execute_udf(process, udf, data, dimension = dimension, context = context)
    t2 = time.time() # End benchmark
    #print('end: ', udf, ' at ', time.strftime('%X %x %Z')) #t2

    # Print result and benchmark
    print('  Time elapsed: %s' % (t2 - t1))
    #save_result(result, save_file)
    result.to_netcdf(save_file)

In [7]:
print(time.strftime('%X %x %Z'), ' reduce_dimension bfast start')
run('reduce_dimension', udf_file, data = dataset, dimension = dimension, context = context)
print(time.strftime('%X %x %Z'), ' reduce_dimension bfast done')

11:03:30 11/08/22 UTC  reduce_dimension bfast start


R[write to console]: Error: .onLoad failed in loadNamespace() for 'lubridate', details:
  call: system(paste(which, shQuote(names[i])), intern = TRUE, ignore.stderr = TRUE)
  error: cannot popen '/usr/bin/which 'timedatectl' 2>/dev/null', probable reason 'Cannot allocate memory'



RRuntimeError: Error: .onLoad failed in loadNamespace() for 'lubridate', details:
  call: system(paste(which, shQuote(names[i])), intern = TRUE, ignore.stderr = TRUE)
  error: cannot popen '/usr/bin/which 'timedatectl' 2>/dev/null', probable reason 'Cannot allocate memory'


In [28]:
# test.nc: 78 sec
# vaja.nc: runs out of memory after 1h15 (on 96GB RAM)

'../00_data_local/vaja.nc/openEO.nc'